In [1]:
# Take the zipped file and extract the 10 text files it contains.
# Two of the ten files contain plagiarised content from one of the other files.
# In one it is a single sentence, in the other an entire paragraph.
# Write a program to analyse the 11 texts for textual content and then use comparison to identify 
# the plagiarised sections.
#
# Tips:
# 1. Remove the punctuation using a linear replacement and converting the sentences into single lines of text. 
#    Also convert everything to upper or lower case.
# 2. Analyse the files to create a word list (a hashed dictionary) replacing the words with numerical values 
#    of the word positions in the list.
# 3. Linked lists are the ideal mechanism for creating this dictionary
# 4. Compare the numerical sentence sequences to identify the copied text and the source and destination files 
#   for each copied element.
#
# The final step may be quite a lengthy process. 
# I’d be interested to know the computation time for your particular hardware.

In [90]:
# https://docs.python.org/2/tutorial/inputoutput.html
# https://docs.python.org/3/library/string.html
# https://docs.python.org/3.4/library/stdtypes.html

import os
import string
import itertools

# word dictionary
wordDict = {} #key: word, value: counter
counter = 0
translateDict = {} #key: counter, value: word

# word list array
#wordListArray = []

# dictionary to store the numeric word lists
numericDic = {} #key: filename, value: wordList

# read txt file
def readTxt(i):
    # read text file
    f = open('Lab3.2/%s.txt' % i , 'r')
    print ('## file %s read ##' %i)
    return f

# write file to disk
def writeFile(i):
    f = open('Lab3.2/numerical/%s.txt' % i , 'w')
    return f

# close file
def closeFile(f):
    f.close()
    
# remove punctuation from string
# http://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
def removePunctuation(s):
    return s.translate(string.maketrans("",""), string.punctuation)

print ('#### START CREATING WORD DICTIONARY ####')
# read all files
for filename in os.listdir('Lab3.2'):
    fn = filename[:2]
    try:
        # read file
        fr = readTxt(fn)
        # create word dictionary
        for line in fr:
            # convert string to lowercase and renove punctuation
            new_line =  removePunctuation(line.lower())
            splitted_line = new_line.split()
            #print split_line
            # split line to words
            for word in splitted_line:
                #print word
                # check word against wordDict
                if word not in wordDict:
                    # add word to wordDict
                    wordDict[word] = counter
                    translateDict[counter] = word
    #                print ("'{}' added to wordDict with value {}".format(word,counter))
                    counter += 1
    #            else:
    #                print ("'{}' already in wordDict".format(word))
        # close file
        closeFile(fr)
    except:
        print ('    file error: {}'.format(filename))

                
print ('#### wordDictionary created with {} words. ####'.format(len(wordDict)))
#print wordDict.values()

# substitute words with numbers in each file
# linked list as new text files
# http://stackoverflow.com/questions/280243/python-linked-list

print ('#### START WORD TO NUMBER CONVERSION ####')
# read all files
for filename in os.listdir('Lab3.2'):
    fn = filename[:2]
    # create empty sentence list
    sentenceList= []
    try:
        # open read file
        fr = readTxt(fn)
        # open write file
#        fw = writeFile(fn)
        for line in fr:
            # create empty word list for each sentence
            wordList = []
            # convert string to lowercase and renove punctuation
            new_line =  removePunctuation(line.lower())
            splitted_line = new_line.split()
            # split line to words
            for word in splitted_line:
                #get value for current word
                value = wordDict[word]
                # save value to wordList
                wordList.append(value)
            sentenceList.append(wordList)
        # add wordList for current file to numericDic
        # key: filename, value: wordList
        numericDic[fn] = sentenceList
        
#        # write wordList to file
#        for item in wordList:
#            fw.write("%s\n" % item)
        # close files
        closeFile(fr)
#        closeFile(fw)
        #print ('wordList: {}'.format(wordList))
    except:
        print ('    file error: {}'.format(filename))       
print ('#### numericDic created for {} files. ####'.format(len(numericDic)))
#print ('numericDic:')
#print numericDic

# compare the sequences in the linked lists
# define threshold for senteces
globalThreshold = 0.9 #percentage

# compare two lists (including order of words, works only for sentences with the equal size lists)
# http://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches
def listCompare(x, y):
    #if (tuple(x) == tuple(y)):
     #   print ('tuple(x): {}'.format(tuple(x)))
    return [i for i, j in zip(x, y) if i == j]

def getBestMatches(longer, shorter, longerLen, shorterLen):
    matches = []
    currentMatches = []
    diff = longerLen - shorterLen
    #print ('shorter sentence: {}'.format(shorter))
    for i in xrange(diff+1):  #exclusive max
        currentMatches = listCompare(longer[i:i+shorterLen], shorter)
        #print ('longer sentence seq: {} with length: {}'.format(seq, longerLen))
        if (len(currentMatches)>len(matches)):
            matches = currentMatches
    return matches    

print ('#### START FINDING PLAGIARISED SECTIONS ####')


#loop through all files
#for outerItem in iter(numericDic):
#    for innerItem in iter(numericDic):
        # do not compare file with itself
#        if (outerItem != innerItem):   

for outerItem,innerItem in itertools.combinations(numericDic,2):
    for outerSentence in numericDic[outerItem]:
        for innerSentence in numericDic[innerItem]:
            # reset matches and percentage
            matches = []
            percentage = 0
            # get sentence length
            osLen = len(outerSentence)
            isLen = len(innerSentence)
            # check against sentence length, both sentences should have at least 3 words
            if (osLen > 2 and isLen > 2):
                if (osLen == isLen):
                    #print ('same length')
                    matches = listCompare(innerSentence, outerSentence)
                    percentage = len(matches) / float(osLen)
                    #print percentage
                elif (osLen > isLen):
                    #print ('osLen larger')
                    matches = getBestMatches(outerSentence, innerSentence, osLen, isLen)
                    # percentage based on shorter sentence; isLen
                    percentage = len(matches) / float(isLen)
                    #print percentage
                else:
                    #print ('isLen larger')
                    matches = getBestMatches(innerSentence, outerSentence, isLen, osLen)
                    # percentage based on shorter sentence: osLen
                    percentage = len(matches) / float(osLen)
                    #print percentage
            #else: # one of the sentences is empty
                #print ('zero length')
            if (percentage >= globalThreshold):
                # translate number sequences back to words
                translatedSentence = []
                firstSentence = []
                secondSentence = []
                for number in matches:
                    translatedSentence.append(translateDict[number])
                for number in outerSentence:
                    firstSentence.append(translateDict[number])
                for number in innerSentence:
                    secondSentence.append(translateDict[number])
                print ('###')
                print ('NEW MATCH FOUND with {}% for {}'.format(percentage*100, matches))
                print ('    {}'.format(translatedSentence))
                print ('  1. numer sequence in file {}: {}'.format(outerItem, outerSentence))
                print ('    {}'.format(firstSentence))
                print ('  2. number sequence in file {}: {}'.format(innerItem, innerSentence))
                print ('    {}'.format(secondSentence))   
        
        
print ('#### DONE ####')       





#### START CREATING WORD DICTIONARY ####
    file error: .DS_Store
## file 01 read ##
## file 02 read ##
## file 03 read ##
## file 04 read ##
## file 05 read ##
## file 06 read ##
## file 07 read ##
## file 08 read ##
## file 09 read ##
## file 10 read ##
    file error: untitled folder
#### wordDictionary created with 4099 words. ####
#### START WORD TO NUMBER CONVERSION ####
    file error: .DS_Store
## file 01 read ##
## file 02 read ##
## file 03 read ##
## file 04 read ##
## file 05 read ##
## file 06 read ##
## file 07 read ##
## file 08 read ##
## file 09 read ##
## file 10 read ##
    file error: untitled folder
#### numericDic created for 10 files. ####
#### START FINDING PLAGIARISED SECTIONS ####
###
NEW MATCH FOUND with 100.0% for [200, 540, 229, 100]
    ['go', 'through', 'with', 'it']
  1. numer sequence in file 02: [200, 540, 229, 100]
    ['go', 'through', 'with', 'it']
  2. number sequence in file 03: [338, 57, 2, 584, 611, 200, 540, 229, 100]
    ['decided', 'that', '